In [10]:
from IPython.display import display
import cv2

In [11]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from tensorflow.python.client import device_lib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

#distance measure tools
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model
import os
import pathlib
# # List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
# # If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
# PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
# TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
# TEST_IMAGE_PATHS

In [12]:
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

In [13]:
model_name = 'faster_rcnn_inception_v2_coco_2018_01_28'
#model_name = 'ssdlite_mobilenet_v2_coco_2018_05_09'
#model_name = 'mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [14]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [15]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [16]:
def show_inference_v1(model, image_np):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  
  #   image_np = np.array(Image.open(image_path))
#   print(image_np.shape)
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)   
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  image_np = measuring_distance(image_np,output_dict)
  #image_np = measuring_distance_template(image_np,output_dict)
  return Image.fromarray(image_np)

#   display(Image.fromarray(image_np))

In [17]:
def measuring_distance(img,output_dict):
    # load the image, convert it to grayscale, and blur it slightly
    image = img
    orig = image.copy()
    img_width = 1280
    img_height = 720
    refObj = None
    # loop over the contours individually
    flag = 0;
    bag = 0;
    for c in output_dict['detection_boxes']:
        if output_dict['detection_scores'][flag] > 0.5 and output_dict['detection_classes'][flag] == 33:
            bag = 1
            # box
            ymin = c[0]
            xmin = c[1]
            ymax = c[2]
            xmax = c[3]
            left = xmin * img_width
            right = xmax * img_width
            top = ymin * img_height
            bottom = ymax * img_height
            box = [[left,top],[left,bottom],[right,top],[right,bottom]]
            box = np.array(box, dtype="int")
            #cv2.drawContours(orig, [box.astype("int")], -1, (255, 255, 255), 10)
            box = perspective.order_points(box)
            #cv2.drawContours(orig, [box.astype("int")], -1, (0, 0, 0), 10)
            # compute the center of the bounding box
            cX = np.average(box[:, 0])
            cY = np.average(box[:, 1])
            # if this is the first contour we are examining (i.e.,
            # the left-most contour), we presume this is the
            # reference object
            if refObj is None:
                # unpack the ordered bounding box, then compute the
                # midpoint between the top-left and top-right points,
                # followed by the midpoint between the top-right and
                # bottom-right
                (tl, tr, br, bl) = box
                (tlblX, tlblY) = midpoint(tl, bl)
                (trbrX, trbrY) = midpoint(tr, br)
                # compute the Euclidean distance between the midpoints,
                # then construct the reference object
                D = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
                refObj = (box, (cX, cY), D / img_width)
        flag = flag +1
    flag = 0
    warning = 1
    for c in output_dict['detection_boxes']:
        # box
        if output_dict['detection_scores'][flag] > 0.5 and output_dict['detection_classes'][flag] == 1 and bag == 1:
            ymin = c[0]
            xmin = c[1]
            ymax = c[2]
            xmax = c[3]
            left = xmin * img_width
            right = xmax * img_width
            top = ymin * img_height
            bottom = ymax * img_height
            box = [[left,top],[right,top],[left,bottom],[right,bottom]]
            box = np.array(box, dtype="int")
            box = perspective.order_points(box)
            # compute the center of the bounding box
            cX = np.average(box[:, 0])
            cY = np.average(box[:, 1])
            # draw the contours on the image
            cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
            cv2.drawContours(orig, [refObj[0].astype("int")], -1, (0, 255, 0), 2)
            # stack the reference coordinates and the object coordinates
            # to include the object center
            refCoords = np.vstack([refObj[1]])
            objCoords = np.vstack([(cX, cY)])
            # loop over the original points
            for ((xA, yA), (xB, yB)) in zip(refCoords, objCoords):
                # draw circles corresponding to the current points and
                # connect them with a line
                cv2.circle(orig, (int(xA), int(yA)), 5, (255, 255, 0), -1)
                cv2.circle(orig, (int(xB), int(yB)), 5, (255, 255, 0), -1)
                cv2.line(orig, (int(xA), int(yA)), (int(xB), int(yB)),
                    (255, 255, 0), 2)
                # compute the Euclidean distance between the coordinates,
                # and then convert the distance in pixels to distance in
                # units
                D = dist.euclidean((xA, yA), (xB, yB)) / refObj[2]
                D = (D/10)/100
                (mX, mY) = midpoint((xA, yA), (xB, yB))
                cv2.putText(orig, "{:.1f}m".format(D), (int(mX), int(mY - 10)),cv2.FONT_HERSHEY_SIMPLEX, 0.55, (240, 0, 159), 2)
                if D < 4 :
                    warning = 0
        flag = flag+1
    if warning == 1 and bag == 1:
        cv2.putText(orig, "Warning!!!".format(D), (int(img_width/2-100), int(img_height/2)),cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)
    return orig

In [18]:

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('changyou4_6iq48i6b_XpBg.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
center = (w / 2, h/ 2)
# Read until video is completed
counter = 0

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
name = 'demo' + '.mp4'
out = cv2.VideoWriter(name, fourcc, 20.0, (1280,720))
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  

  if ret == True:
#     print(type(frame))
#     M = cv2.getRotationMatrix2D(center, 270, 1)
#     img2 = cv2.warpAffine(frame, M, (w, h))
    # Display the resulting frame
    if counter%1 == 0: 
        img2 = cv2.resize(frame,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        img_result =  np.array(show_inference_v1(detection_model, img2))
        #final_img = measuring_distance(img_result)
        cv2.imshow('Frame',img_result)
        counter = 0
        out.write(img_result)
        # Press Q on keyboard to  exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
          break
  # Break the loop
    counter +=1
  else: 
    break

# When everything done, release the video capture object
cap.release()
out.release()
# Closes all the frames
cv2.destroyAllWindows()